# SNEWS Coincidence System Tests

In this notebook, we collected some inputs to test features at the coincidence system.<br>
To test, one needs to run the coincidence on a local machine and trigger the commands here. Which should be executed at the coincidence system and the logs can be displayed.

After starting the coincidence system (and, if desired, the feedback script).
```bash
user$: snews_cs run-coincidence --firedrill
user$: snews_cs run-feedbacks --firedrill
```

_Note_:<br>
If you haven't initiated a mongo instance, you also need to start that before running the coincidence system, as the messages are fed to mongo. <br>
```bash
user$: sudo service mongodb start
```

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from snews_pt.snews_pub import SNEWSTiersPublisher
from snews_cs.core.logging import log_file

/mnt/c/Users/bj7780/Desktop/Kara/GitHub/SNEWS/SNEWS_Publishing_Tools/snews_pt/__init__.py:23: UserWarning: You are using default detector name "TEST"
Please change this by snews_pt.snews_pt_utils.set_name()
  warnings.warn(warning_text, UserWarning)


In [2]:
# the logs of "today" will be saved here
log_file

'/mnt/c/Users/bj7780/Desktop/Kara/GitHub/SNEWS/SNEWS_Coincidence_System/snews_cs/core/../../logs/2022-11-21.log'

Once the `snews_cs run-coincidence` is executed, the log file is created

In [3]:
!cat $log_file

2022-11-21 13:14:08 on iap-nb-034
  DEBUG [snews_cs.snews_coinc_v2] Initializing CoincDecider

2022-11-21 13:14:08 on iap-nb-034
  INFO [snews_cs.snews_hb] 	> Heartbeat Instance is created.
2022-11-21 13:14:08 on iap-nb-034
  INFO [snews_cs.snews_hb] 	> Heartbeats are stored in /mnt/c/Users/bj7780/Desktop/Kara/GitHub/SNEWS/SNEWS_Coincidence_System/snews_cs/../beats.


### Test coincidence 
Send two messages from two detectors with short time interval so it triggers an alert.

In [4]:
time1 = "2023-06-14T12:45:45.100000" #datetime.strptime("2023-06-14T12:45:45.100000", '%Y-%m-%dT%H:%M:%S.%f')
time2 = "2023-06-14T12:45:49.100000" #datetime.strptime("2023-06-14T12:45:49.100000", '%Y-%m-%dT%H:%M:%S.%f')

msg1 = SNEWSTiersPublisher(detector_name="XENONnT",
                           neutrino_time=time1,
                           is_test=True)

msg2 = SNEWSTiersPublisher(detector_name="DS-20K",
                           neutrino_time=time2,
                           is_test=True)

Message Generated for CoincidenceTier
Message Generated for CoincidenceTier


In [5]:
for msg in [msg1, msg2]:
    msg.send_to_snews()

----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-firedrill
_id                :19_CoincidenceTier_2022-11-21T13:14:22.639583
detector_name      :XENONnT
machine_time       :2022-11-21T13:14:22.639583
neutrino_time      :2023-06-14T12:45:45.100000
p_val              :None
meta               :
	is_test            :True
schema_version     :1.2.0
sent_time          :2022-11-21T13:14:22.639583
----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-firedrill
_id                :18_CoincidenceTier_2022-11-21T13:14:22.661826
detector_name      :DS-20K
machine_time       :2022-11-21T13:14:22.661826
neutrino_time      :2023-06-14T12:45:49.100000
p_val              :None
meta               :
	is_test            :True
schema_version     :1.2.0
sent_time          :2022-11-21T13:14:22.661826


In [6]:
!cat $log_file

2022-11-21 13:14:08 on iap-nb-034
  DEBUG [snews_cs.snews_coinc_v2] Initializing CoincDecider

2022-11-21 13:14:08 on iap-nb-034
  INFO [snews_cs.snews_hb] 	> Heartbeat Instance is created.
2022-11-21 13:14:08 on iap-nb-034
  INFO [snews_cs.snews_hb] 	> Heartbeats are stored in /mnt/c/Users/bj7780/Desktop/Kara/GitHub/SNEWS/SNEWS_Coincidence_System/snews_cs/../beats.
2022-11-21 13:14:28 on iap-nb-034
  DEBUG [snews_cs.cs_remote_commands] 	> Handling message..

2022-11-21 13:14:28 on iap-nb-034
  DEBUG [snews_cs.cs_remote_commands] **************************************** LOGS FOR THE -TEST- MESSAGE
{
    "_id": "19_CoincidenceTier_2022-11-21T13:14:22.639583",
    "detector_name": "XENONnT",
    "machine_time": "2022-11-21T13:14:22.639583",
    "meta": {
        "is_test": true
    },
    "neutrino_time": "2023-06-14T12:45:45.100000",
    "p_val": null,
    "schema_version": "1.2.0",
    "sent_time": "2022-11-21T13:14:22.639583"
}
2022-11-21 13:14:28 on iap-nb-034
  DEBUG [snews_cs.cs_re

### Try retraction

In [7]:
msg3 = SNEWSTiersPublisher(detector_name="DS-20K",
                           neutrino_time=time2,
                           n_retract_latest=1,
                           is_test=True)
msg3.messages[0]['is_test'] = True
msg3.send_to_snews()

Message Generated for CoincidenceTier
Message Generated for Retraction
----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-firedrill
_id                :18_CoincidenceTier_2022-11-21T13:14:43.101419
detector_name      :DS-20K
machine_time       :2022-11-21T13:14:43.101419
neutrino_time      :2023-06-14T12:45:49.100000
p_val              :None
meta               :
	is_test            :True
schema_version     :1.2.0
sent_time          :2022-11-21T13:14:43.101419
is_test            :True
----------------------------------------------------------------
Sending message to Retraction on kafka://kafka.scimma.org/snews.experiments-firedrill
IT'S OKAY, WE ALL MAKE MISTAKES
_id                :18_Retraction_2022-11-21T13:14:43.101419
detector_name      :DS-20K
machine_time       :2022-11-21T13:14:43.101419
which_tier         :None
N_retract_latest   :1
retraction_reason  :None
meta               :
	is_t

In [8]:
!cat $log_file

2022-11-21 13:14:08 on iap-nb-034
  DEBUG [snews_cs.snews_coinc_v2] Initializing CoincDecider

2022-11-21 13:14:08 on iap-nb-034
  INFO [snews_cs.snews_hb] 	> Heartbeat Instance is created.
2022-11-21 13:14:08 on iap-nb-034
  INFO [snews_cs.snews_hb] 	> Heartbeats are stored in /mnt/c/Users/bj7780/Desktop/Kara/GitHub/SNEWS/SNEWS_Coincidence_System/snews_cs/../beats.
2022-11-21 13:14:28 on iap-nb-034
  DEBUG [snews_cs.cs_remote_commands] 	> Handling message..

2022-11-21 13:14:28 on iap-nb-034
  DEBUG [snews_cs.cs_remote_commands] **************************************** LOGS FOR THE -TEST- MESSAGE
{
    "_id": "19_CoincidenceTier_2022-11-21T13:14:22.639583",
    "detector_name": "XENONnT",
    "machine_time": "2022-11-21T13:14:22.639583",
    "meta": {
        "is_test": true
    },
    "neutrino_time": "2023-06-14T12:45:45.100000",
    "p_val": null,
    "schema_version": "1.2.0",
    "sent_time": "2022-11-21T13:14:22.639583"
}
2022-11-21 13:14:28 on iap-nb-034
  DEBUG [snews_cs.cs_re

Two issues I've encountered with the retraction so far:
 - It starts by converting the neutrino times to datetime obj, and silently crashes if there are no neutrino times. We said that by default it should look at the last message of a given detector. so it should work without a neutrino time.
 - two:
 ```
 UPDATING MESSAGE FROM: DS-20K                                                                                                                                                                                   [0]                                                                                                                                                                                                         drop() got an unexpected keyword argument 'column'  
 ```